In [1]:
!git clone --single-branch --branch u5_preview https://github.com/WongKinYiu/PyTorch_YOLOv4.git

Cloning into 'PyTorch_YOLOv4'...
remote: Enumerating objects: 141, done.
remote: Total 141 (delta 0), reused 0 (delta 0), pack-reused 141
Receiving objects: 100% (141/141), 1012.90 KiB | 1.94 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [2]:
%cd PyTorch_YOLOv4
!pip install -r requirements.txt

/kaggle/working/PyTorch_YOLOv4
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-a8iy0deb
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-a8iy0deb
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
     |████████████████████████████████| 20.3 MB 31.3 MB/s eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=272170 sha256=a79a1213182c03eccb163bc639f97b789f8630687dd60a7e76544abb748706f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-y8_uo_qq/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [3]:
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.1 (Tesla P100-PCIE-16GB)


In [4]:
from distutils.dir_util import copy_tree
copy_tree('/kaggle/input/', '/kaggle/working')

['/kaggle/working/dataset/Dataset/labels/validation/3c511a8c88dc0d13.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/81574c970805e79e.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/12b9323540615016.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/71d7fec27a4a734c.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/b40c2394b5ec25b7.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/200907829e4fbe95.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/bd27b640aa21d82d.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/ec724ef6386c46fd.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/e9b6333284bec5b5.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/82d0cc0429c0c899.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/5e6ff0ca93baad99.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/6022a450df278025.txt',
 '/kaggle/working/dataset/Dataset/labels/validation/a9371dcd7d3087d9.txt',
 '/kaggle/working/dataset

In [5]:
# Create .yaml file 
import yaml

data_yaml = dict(
    train = '/kaggle/working/dataset/Dataset/images/train/',
    val = '/kaggle/working/dataset/Dataset/images/validation/',
    nc = 9,
    names = ['Bicycle','Car','Handgun','Knife','Luggage and bags','Motorcycle','Person','Shotgun','Truck']
)

with open('./data/data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat ./data/data.yaml

{names: [Bicycle, Car, Handgun, Knife, Luggage and bags, Motorcycle, Person, Shotgun,
    Truck], nc: 9, train: /kaggle/working/dataset/Dataset/images/train/, val: /kaggle/working/dataset/Dataset/images/validation/}


In [6]:
with open('./models/yolov4-tiny.yaml') as fp:
    data = yaml.load(fp)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [7]:
data['nc']=9
with open('./models/yolov4-tiny.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=True)
    
%cat './models/yolov4-tiny.yaml'

{anchors: [[23, 27, 37, 58, 81, 82], [81, 82, 135, 169, 344, 319]], backbone: [[-1,
      1, Conv, [32, 3, 2]], [-1, 1, Conv, [64, 3, 2]], [-1, 1, Conv, [64, 3, 1]],
    [-1, 1, VoVCSP, [64]], [[-2, -1], 1, Concat, [1]], [-1, 1, MP, [2]], [-1, 1, Conv,
      [128, 3, 1]], [-1, 1, VoVCSP, [128]], [[-2, -1], 1, Concat, [1]], [-1, 1, MP,
      [2]], [-1, 1, Conv, [256, 3, 1]], [-1, 1, VoVCSP, [256]], [[-2, -1], 1, Concat,
      [1]], [-1, 1, MP, [2]], [-1, 1, Conv, [512, 3, 1]]], depth_multiple: 1.0, head: [
    [-1, 1, Conv, [256, 1, 1]], [-1, 1, Conv, [512, 3, 1]], [-1, 1, nn.Conv2d, [na
          * (nc + 5), 1, 1]], [-2, 1, Conv, [256, 1, 1]], [-1, 1, nn.Upsample, [None,
        2, nearest]], [[-1, 11], 1, Concat, [1]], [-1, 1, Conv, [256, 3, 1]], [-1,
      1, nn.Conv2d, [na * (nc + 5), 1, 1]], [[], 1, Detect, [nc, anchors]]], nc: 9,
  width_multiple: 1.0}


In [8]:
!add-apt-repository ppa:ubuntu-toolchain-r/test -y
!apt-get update
!apt-get upgrade libstdc++6 -y

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:3 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [5385 B]   
Get:4 http://ppa.launchpad.net/ubuntu-toolchain-r/test/ubuntu bionic InRelease [20.8 kB]
Get:5 https://packages.cloud.google.com/apt google-fast-socket InRelease [5405 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:7 http://packages.cloud.google.com/apt cloud-sdk-bionic InRelease [6786 B] 
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Get:9 https://packages.cloud.google.com/apt google-fast-socket/main amd64 Packages [429 B]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2835 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease

In [10]:
!pip install torch==1.7.0+cu110 torchvision==0.8.1+cu110 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 1137.1 MB 37.2 MB/s eta 0:00:01    |██████████████▎                 | 508.0 MB 10.7 MB/s eta 0:00:59     |██████████████▌                 | 515.4 MB 10.7 MB/s eta 0:00:59     |███████████████                 | 536.9 MB 10.7 MB/s eta 0:00:57     |█████████████████████▎          | 755.8 MB 67.4 MB/s eta 0:00:06     |█████████████████████▍          | 761.6 MB 67.4 MB/s eta 0:00:06     |█████████████████████▌          | 764.8 MB 67.4 MB/s eta 0:00:06:00:06     |█████████████████████▋          | 768.1 MB 4.6 MB/s eta 0:01:21     |██████████████████████          | 782.5 MB 4.6 MB/s eta 0:01:18     |██████████████████████          | 783.9 MB 4.6 MB/s eta 0:01:18     |██████████████████████          | 785.2 MB 4.6 MB/s eta 0:01:17     |██████████████████████▌         | 800.7 MB 49.5 MB/s eta 0:00:07     |██████████████████████▊         | 805.9 MB 49.5 MB/s eta 0:00:07     |█████████████

In [11]:
!python train.py --img-size 320 320 --batch-size 16 --epochs 120 --cfg yolov4-tiny.yaml --data data.yaml --weights '/kaggle/working/weighttinyyolov4/best_yolov4-tiny2.pt' --name yolov4-tiny

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='./models/yolov4-tiny.yaml', data='./data/data.yaml', device='', epochs=120, evolve=False, img_size=[320, 320], multi_scale=False, name='yolov4-tiny', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='/kaggle/working/weighttinyyolov4/best_yolov4-tiny2.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/

              from  n    params  module                                  arg

In [12]:
!zip -r /kaggle/working/file3.zip /kaggle/working/PyTorch_YOLOv4

  adding: kaggle/working/PyTorch_YOLOv4/ (stored 0%)
  adding: kaggle/working/PyTorch_YOLOv4/hubconf.py (deflated 74%)
  adding: kaggle/working/PyTorch_YOLOv4/runs/ (stored 0%)
  adding: kaggle/working/PyTorch_YOLOv4/runs/Oct26_23-58-04_e9e7649b244dyolov4-tiny/ (stored 0%)
  adding: kaggle/working/PyTorch_YOLOv4/runs/Oct26_23-58-04_e9e7649b244dyolov4-tiny/events.out.tfevents.1635292688.e9e7649b244d.56921.0 (deflated 65%)
  adding: kaggle/working/PyTorch_YOLOv4/runs/Oct26_23-56-09_e9e7649b244dyolov4-tiny/ (stored 0%)
  adding: kaggle/working/PyTorch_YOLOv4/runs/Oct26_23-56-09_e9e7649b244dyolov4-tiny/events.out.tfevents.1635292572.e9e7649b244d.56818.0 (deflated 5%)
  adding: kaggle/working/PyTorch_YOLOv4/results_yolov4-tiny.txt (deflated 76%)
  adding: kaggle/working/PyTorch_YOLOv4/weights/ (stored 0%)
  adding: kaggle/working/PyTorch_YOLOv4/weights/put your weights file here.txt (deflated 43%)
  adding: kaggle/working/PyTorch_YOLOv4/weights/last_yolov4-tiny.pt (deflated 8%)
  adding: ka